# Assess predictions on Stanford Question Answering Dataset (SQuAD) with a huggingface question answering model

This notebook demonstrates the use of the `responsibleai` API to assess a huggingface question answering model on the SQuAD dataset (see https://huggingface.co/datasets/squad for more information about the dataset). It walks through the API calls necessary to create a widget with model analysis insights, then guides a visual analysis of the model.

* [Launch Responsible AI Toolbox](#Launch-Responsible-AI-Toolbox)
    * [Load Model and Data](#Load-Model-and-Data)
    * [Create Model and Data Insights](#Create-Model-and-Data-Insights)

## Launch Responsible AI Toolbox

The following section examines the code necessary to create datasets and a model. It then generates insights using the `responsibleai` API that can be visually analyzed.

### Prepare

To run this notebook, we need to install the following packages:

```
raiutils
raiwidgets
datasets
transformers
responsibleai_text
torch
```

Run the following command to load the spacy pipeline:

```bash
python -m spacy download en_core_web_sm
```

### Load Model and Data
*The following section can be skipped. It loads a dataset and trains a model for illustrative purposes.*

First we import all necessary dependencies

In [ ]:
import datasets
import pandas as pd
from transformers import pipeline

Next we load the SQuAD dataset from huggingface datasets

In [ ]:
dataset = datasets.load_dataset("squad", split="train")
dataset

Reformat the dataset to be a pandas dataframe with three columns: context, questions and answers

In [ ]:
questions = []
context = []
answers = []
for row in dataset:
    context.append(row['context'])
    questions.append(row['question'])
    answers.append(row['answers']['text'][0])

In [ ]:
data = pd.DataFrame({'context': context, 'questions': questions, 'answers': answers})
data = data.sample(frac=1.0, random_state=42).reset_index(drop=True)
data.head()

Fetch a huggingface question answering model

In [ ]:
# load the question-answering model
pipeline_model = pipeline('question-answering')
test_size = 5

train_data = data
test_data = data[:test_size]

See an example of the model's predictions

In [ ]:
def get_answer(dataset, idx):
    model_output = pipeline_model(question=dataset['questions'][idx], 
                                  context=dataset['context'][idx])
    pred = model_output['answer']
    return pred

def check_answer(dataset, idx):
    pred = get_answer(dataset, idx)
    print('Question  : ', dataset['questions'][idx])
    print('Answer    : ', dataset['answers'][idx])
    print('Predicted : ', pred)
    print('Correct   : ', pred == dataset['answers'][idx])

check_answer(test_data, 0)


### Create Model and Data Insights

In [ ]:
from responsibleai_text import RAITextInsights, ModelTask
from raiwidgets import ResponsibleAIDashboard

To use Responsible AI Dashboard, initialize a RAITextInsights object upon which different components can be loaded.

RAITextInsights accepts the model, the test dataset, the classes and the task type as its arguments.

In [ ]:
rai_insights = RAITextInsights(pipeline_model, test_data, "answers",
                               task_type=ModelTask.QUESTION_ANSWERING)

Add the components of the toolbox for model assessment.

In [ ]:
rai_insights.error_analysis.add()
rai_insights.explainer.add()

Once all the desired components have been loaded, compute insights on the test set.

In [ ]:
rai_insights.compute()

Finally, visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab.

In [ ]:
ResponsibleAIDashboard(rai_insights)